# Model surface GHI using Sentinel-2, Sentinel-5P, Landsat, DSM and LUT 

Requirements: 

- A GOOGLE cloud project linked to your GOOGLE account


### My project 

Google account: gelieza.gk@gmail.com


cloud project ID: sample-project-452812 


cloud project name: Sample Project


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ee
import geemap
import math
import datetime
import calendar
import pytz
import pvlib

In [2]:
# Authorization and initialization of the GEE
# OBS: You must create our own project on Google
# Authenticate Earth Engine
ee.Authenticate()
#
# Initialize Earth Engine
my_project_name = 'sample-project-452812' # use here the name of your own project on Google
ee.Initialize(project=my_project_name)

gio: https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ecF8cH9A-8O1nB4dPS54tffkMzQkROdCGBgxbauh3Kg&tc=OD0RjeOJSp3z_OvV9EK3bF6XyAXd7aUVmdt2jgPDKuo&cc=nMTxWIIgEgDdYk69soCdl_bjGvel4agpr_4PaEfsnWU: Operation not supported



Successfully saved authorization token.


In [3]:
# Convert degree to radian
def deg2rad(deg):
    return deg * math.pi / 180

# Define the center of the bounding box (Bergen, Norway)
CENTER_LAT = 60.39
CENTER_LON = 5.33

# Approximate degree adjustments for 100km x 100km box
DEG_LAT_TO_KM = 111.412  # 1 degree latitude at 60° converted to km (https://en.wikipedia.org/wiki/Latitude)
DEG_LON_TO_KM = 111.317 * math.cos(deg2rad(CENTER_LAT))  # 1 degree longitude converted to km
LAT_OFFSET = 12.5 / DEG_LAT_TO_KM  # ~10km north/south
LON_OFFSET = 12.5 / DEG_LON_TO_KM  # ~10km east/west (varies with latitude, approximation)

# Define the bounding box
BBOX = {
    "north": CENTER_LAT + LAT_OFFSET,
    "south": CENTER_LAT - LAT_OFFSET,
    "west": CENTER_LON - LON_OFFSET,
    "east": CENTER_LON + LON_OFFSET
}

print(BBOX)

# Geometry Rectangle of Form minLng, minLat, maxLng, maxLat
bergen_roi = ee.Geometry.Rectangle([BBOX["west"], BBOX["south"], BBOX["east"], BBOX["north"]])

Map = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=10)

# Add the geometry to the map
Map.addLayer(bergen_roi, {"color": "red"}, "Bergen ROI")

# Display the weather stations
stations = {
    #"Fana - Stend": (60.261870, 5.302989),
    "Flesland Bergen": (60.292792, 5.222689),
    "Florida": (60.3833, 5.3333)
}

for name, (lat, lon) in stations.items():
    point = ee.Geometry.Point([lon, lat])
    Map.addLayer(point, {"color": "blue"}, name)
    
# Add elevation map from Hoydedata National Elevation Project 1m
dsm = ee.Image("projects/sample-project-452812/assets/bergen_dsm_1m_zip").clip(bergen_roi)

elevation_vis = {
    'min': 0,
    'max': 800,
    'palette': ['blue', "green", 'brown', 'white']
}

Map.addLayer(dsm, elevation_vis, 'Elevation (DSM 1m)')

# Display the map
Map

{'north': 60.50219617276416, 'south': 60.27780382723584, 'west': 5.10273148294384, 'east': 5.55726851705616}


Map(center=[60.39, 5.33], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [70]:
# Export reprojected image to make it more usable - and check how much runtime this step actually requires
dsm_merc = dsm.reproject(crs='EPSG:3857', scale=1) #scale 10 5m export time, scale 1: 18 m export time

export_task = ee.batch.Export.image.toAsset(
    image=dsm_merc,
    description='Export_Bergen_DSM_3857',
    assetId='projects/sample-project-452812/assets/bergen_dsm_merc_1m',
    region=bergen_roi,
    scale=1,
    crs='EPSG:3857',
    maxPixels=1e13
)

export_task.start()
#TODO: if no success speeding up things with mercator to asset export, try doing everything at coarser scale!
# Try 30 m first and then 10 m

In [85]:
# Merge datasets

# Binary mask for cloud mask
CLD_PRB_THRESH = 50 # turn int data to binary using different thresholds

def add_cloud_mask(image):
    cloud_mask = image.select('probability').gt(CLD_PRB_THRESH).rename('cloud_mask')
    # drop band probability immediately for performance reasons
    return image.addBands(cloud_mask).select(['cloud_mask'])

# Convert cloud base height from m to km (it is stored in km in the LUT)
def convert_cbh_to_km(image):
    cbh_km = image.select('cloud_base_height').divide(1000).rename('cloud_base_height')
    return image.addBands(cbh_km, overwrite=True)

def print_collection_info(collection, name=""):
    """Prints the number of images and unique acquisition dates in a collection."""
    total_size = collection.size().getInfo()
    unique_dates = collection.aggregate_array('system:time_start') \
        .map(lambda t: ee.Date(t).format('YYYY-MM-dd')) \
        .distinct().size().getInfo()
    
    print(f"📦 {name} - Total images: {total_size}, Unique acquisition dates: {unique_dates}")
    return total_size, unique_dates

def print_unique_dates_per_month(image_collection, name=""):
    """
    Prints the number of unique acquisition dates per calendar month for a given Earth Engine ImageCollection.
    
    Args:
        image_collection (ee.ImageCollection): The input image collection.
        name (str): Optional label for the dataset being analyzed.
    """
    # Get all acquisition timestamps
    timestamps = image_collection.aggregate_array('system:time_start')

    # Convert to YYYY-MM-DD strings
    unique_dates = timestamps.map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()

    # Extract month from each unique date
    months = unique_dates.map(lambda d: ee.Number.parse(ee.String(d).slice(5, 7)))  # slice gets MM from YYYY-MM-DD

    # Count occurrences per month
    month_counts = months.reduce(ee.Reducer.frequencyHistogram()).getInfo()

    # Convert and print
    month_distribution = {calendar.month_name[int(k)]: v
                          for k, v in sorted(month_counts.items(), key=lambda x: int(x[0]))}

    print(f"\n📅 Unique acquisition dates per month for {name}:")
    for month, count in month_distribution.items():
        print(f"{month}: {count}")

startDate = "2020-04-01" #"2018-07-06" # first date in S5P
endDate = "2020-05-01" #"2025-07-13" # end date in S5P


# Sentinel-5P cloud properties
s5p = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").filterDate(startDate, endDate).filterBounds(bergen_roi) \
    .select(["cloud_optical_depth", "surface_albedo", "cloud_base_height"]) \
    .map(convert_cbh_to_km)

# Sentinel-2 cloud probability 
s2_cloud = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY") \
    .filterDate(startDate, endDate).filterBounds(bergen_roi).map(add_cloud_mask)
    
# Sentinel-2 Harmonized top of atmosphere
def extract_s2harm_cloud_bits(image):
    qa60 = image.select("QA60")
    
    # Extract Bit 11 (cirrus clouds)
    cirrus = qa60.bitwiseAnd(1 << 11).rightShift(11).rename("is_cirrus")

    # Only select 'is_cirrus' for performance reasons - is roughly identical to the negative of 'is_opaque'
    return image.addBands([cirrus]).select(["is_cirrus"])

# Filter and process Sentinel-2 Harmonized
s2_harm = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterDate(startDate, endDate) \
    .filterBounds(bergen_roi) \
    .select("QA60") \
    .map(extract_s2harm_cloud_bits)

def extract_landsat_cloud_bits(image):
    """
    Extracts cloud_mask and is_cirrus bands from the QA_PIXEL band in Landsat Collection.
    - cloud_mask = 1 if Bit 3 is set (cloud) AND Bit 6 is NOT set (not clear)
    - is_cirrus = Bit 2
    The original QA_PIXEL band is removed.
    """
    qa = image.select("QA_PIXEL")

    # Extract bits
    bit2 = qa.bitwiseAnd(1 << 2).rightShift(2)  # Cirrus
    bit3 = qa.bitwiseAnd(1 << 3).rightShift(3)  # Cloud
    bit6 = qa.bitwiseAnd(1 << 6).rightShift(6)  # Clear

    # cloud_mask = 1 if cloud (bit3==1) AND NOT clear (bit6==0)
    cloud_mask = bit3.And(bit6.Not())

    # Rename and add bands
    return image \
        .addBands(cloud_mask.rename("cloud_mask")) \
        .addBands(bit2.rename("is_cirrus")) \
        .select(["cloud_mask", "is_cirrus"])  # drop QA_PIXEL


# Landsat
l9_T1 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
    .filterDate(startDate, endDate) \
    .filterBounds(bergen_roi) \
    .select('QA_PIXEL') \
    .map(extract_landsat_cloud_bits)
l9_T2 = ee.ImageCollection("LANDSAT/LC09/C02/T2_L2") \
    .filterDate(startDate, endDate) \
    .filterBounds(bergen_roi) \
    .select('QA_PIXEL') \
    .map(extract_landsat_cloud_bits)
l8_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate(startDate, endDate) \
    .filterBounds(bergen_roi) \
    .select('QA_PIXEL') \
    .map(extract_landsat_cloud_bits)
l8_T2 = ee.ImageCollection("LANDSAT/LC08/C02/T2_L2") \
    .filterDate(startDate, endDate) \
    .filterBounds(bergen_roi) \
    .select('QA_PIXEL') \
    .map(extract_landsat_cloud_bits)
   
l8_9 = l8_T1.merge(l8_T2).merge(l9_T1).merge(l9_T2)


      
# Merge all bands from join result 
def merge_all_bands(join_result):
    primary = ee.Image(join_result.get('primary'))  # S2 cloud + harmonized (already cat-ed)
    secondary = ee.Image(join_result.get('secondary'))  # S5P
    return primary.addBands(secondary)

# For each s2 cloud property image, join with s2 harmonized +- 1 min 
# (they should have the same acquisition timestamps)
min10_millis = 10 * 60 * 1000  # 10 min
max10minDiffFilter = ee.Filter.maxDifference(**{
    'difference': min10_millis,
    'leftField': 'system:time_start',
    'rightField': 'system:time_start'
})

spatialFilter = ee.Filter.intersects(**{
  'leftField': '.geo',
  'rightField': '.geo'})

combined_filter = ee.Filter.And(spatialFilter, max10minDiffFilter)
inner_join = ee.Join.inner()

inner_joined_s2_s2h = inner_join.apply(s2_cloud, s2_harm, combined_filter)

joined_s2 = ee.ImageCollection(inner_joined_s2_s2h.map(merge_all_bands))

# 1 min : ~ 180 unique dates
# 5 min : 6103 combinations, 379 unique dates
# 10 min: 16320 combinations, 954 unique dates (max because this is size of S2 Harmonized)

# Join S2 and S5P
# Now join with Sentinel-5P
min15_millis = 15 * 60 * 1000  # unique dates: 10 min: 192, 15 min: 281 (7-28 per month), 
                                # 30 min: 9335 obs, 542 unique dates(15-55 per month) 
# Average wind speed in Bergen is 6.5 m/s throughout the year (https://de.wikipedia.org/wiki/Windgeschwindigkeit)
# That means clouds can travel ~ 6 km in 15 minutes, 12 in 30 minutes
max15minDiffFilter = ee.Filter.maxDifference(**{
    'difference': min15_millis,
    'leftField': 'system:time_start',
    'rightField': 'system:time_start'
})

combined_filter15min = ee.Filter.And(spatialFilter, max15minDiffFilter)
inner_joined_s2_s5p = inner_join.apply(joined_s2, s5p, combined_filter15min)

# Create merged ImageCollection with all bands
merged_collection_s2_s5p = ee.ImageCollection(inner_joined_s2_s5p.map(merge_all_bands))

# Now join landsat and s5p
# 5 min: 8 images, 10 min: 75, 15 min: 168 images (132 unique dates, 8-22 per month, no obs in nov-jan)
inner_joined_s5p_ls = inner_join.apply(s5p, l8_9, combined_filter15min)

# Create merged ImageCollection with all bands
merged_collection_s5p_ls = ee.ImageCollection(inner_joined_s5p_ls.map(merge_all_bands))

# Merge all into one collection 
merged_all = merged_collection_s2_s5p.merge(merged_collection_s5p_ls)

#print_collection_info(merged_all, "Merged All")
#print_unique_dates_per_month(merged_all, "Merged All")

In [86]:
# Note: Assume equal COD, CBH and albedo values for whole region for now 
# Add image-level mean and mode properties using valid pixels only
def add_image_properties(image):
    # Separate masks for each variable
    albedo_mask = image.select('surface_albedo').gt(0)
    cod_mask = image.select('cloud_optical_depth').gt(0)
    cbh_mask = image.select('cloud_base_height').gt(0)

    # Reduce each variable using only valid pixels
    albedo_mean = image.updateMask(albedo_mask).select('surface_albedo').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bergen_roi,
        scale=1113,
        maxPixels=1e9,
        bestEffort=True
    ).get('surface_albedo')

    cod_mean = image.updateMask(cod_mask).select('cloud_optical_depth').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bergen_roi,
        scale=1113,
        maxPixels=1e9,
        bestEffort=True
    ).get('cloud_optical_depth')

    cbh_mean = image.updateMask(cbh_mask).select('cloud_base_height').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=bergen_roi,
        scale=1113,
        maxPixels=1e9,
        bestEffort=True
    ).get('cloud_base_height')

    # Mode reducer for cloud type
    cloud_type_mode = image.select('is_cirrus').reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=bergen_roi,
        scale=1113,
        maxPixels=1e9,
        bestEffort=True
    ).get('is_cirrus')

    date = ee.Date(image.get('system:time_start'))
    month = date.get('month')
    doy = date.getRelative('day', 'year').add(1)
    hour = date.get('hour')
    minute = date.get('minute')

    # Drop unnecessary bands for performance reasons
    image = image.select(['cloud_mask'])
    
    return image.set({
        'mean_albedo': albedo_mean,
        'mean_cod': cod_mean,
        'mean_cbh': cbh_mean,
        'cloud_type': cloud_type_mode,  # 0 = liquid, 1 = ice
        'month': month,
        'doy': doy,
        'hour': hour,
        'minute': minute
    })

annotated = merged_all.map(add_image_properties) # one month size:122
# Filter for null values and set to default values so LUT will not return null
missing_albedo = annotated.filter(ee.Filter.notNull(['mean_albedo']).Not()).map(lambda img: img.set({'mean_albedo': 0.2}))
annotated = annotated.filter(ee.Filter.notNull(['mean_albedo'])).merge(missing_albedo)

missing_cod = annotated.filter(ee.Filter.notNull(['mean_cod']).Not()).map(lambda img: img.set({'mean_cod': 15.0}))
annotated = annotated.filter(ee.Filter.notNull(['mean_cod'])).merge(missing_cod)

missing_cbh = annotated.filter(ee.Filter.notNull(['mean_cbh']).Not()).map(lambda img: img.set({'mean_cbh': 1.0}))
annotated = annotated.filter(ee.Filter.notNull(['mean_cbh'])).merge(missing_cbh)

missing_cloud_type = annotated.filter(ee.Filter.notNull(['cloud_type']).Not()).map(lambda img: img.set({'cloud_type': 0}))
annotated = annotated.filter(ee.Filter.notNull(['cloud_type'])).merge(missing_cloud_type)

# Set cloud type to 0 = "water" if cod > 30
high_cod = annotated.filter(ee.Filter.gt('mean_cod', 30)).map(lambda img: img.set({'cloud_type': 0}))
annotated = annotated.filter(ee.Filter.lte('mean_cod', 30)).merge(high_cod) # Test successful: same number of images

In [87]:
# Convert to Mercator projection for hillShade computation
dsm_merc = ee.Image("projects/sample-project-452812/assets/bergen_dsm_merc_10m") 

# Loop over merged collection and add band hillshade to each image from zenith angle 
# source: NOAA Global Monitoring Division: General Solar Position Calculations (https://gml.noaa.gov/grad/solcalc/solareqns.PDF)
def add_hillshadow_band(image):
    date = ee.Date(image.get('system:time_start'))

    # Extract UTC hour and day of year
    doy = date.getRelative('day', 'year').add(1)  # DOY = 1-based
    hour = date.get('hour')
    minute = date.get('minute')
    second = date.get('second')
    
    # Note: treat leap years as non-leap years for simplicity (avoid Algorithms.If())
    # Fractional year γ (in radians)
    gamma = ee.Number(2 * math.pi).divide(365).multiply(
        doy.subtract(1).add(hour.subtract(12).divide(24))
    )

    # Equation of time (in minutes)
    eqtime = ee.Number(229.18).multiply(
        ee.Number(0.000075)
        .add(ee.Number(0.001868).multiply(gamma.cos()))
        .subtract(ee.Number(0.032077).multiply(gamma.sin()))
        .subtract(ee.Number(0.014615).multiply((gamma.multiply(2)).cos()))
        .subtract(ee.Number(0.040849).multiply((gamma.multiply(2)).sin()))
    )

    # Solar declination (in radians)
    decl = (
        ee.Number(0.006918)
        .subtract(ee.Number(0.399912).multiply(gamma.cos()))
        .add(ee.Number(0.070257).multiply(gamma.sin()))
        .subtract(ee.Number(0.006758).multiply((gamma.multiply(2)).cos()))
        .add(ee.Number(0.000907).multiply((gamma.multiply(2)).sin()))
        .subtract(ee.Number(0.002697).multiply((gamma.multiply(3)).cos()))
        .add(ee.Number(0.00148).multiply((gamma.multiply(3)).sin()))
    )


    # Time offset in minutes
    time_offset = eqtime.add(ee.Number(CENTER_LON).multiply(4)) 
    # Note: no time-zone awareness because system:time_start is in UTC

    # True solar time (TST), in minutes
    tst = hour.multiply(60).add(minute).add(second.divide(60)).add(time_offset)

    # Hour angle (degrees)
    ha = tst.divide(4).subtract(180)

    # Convert to radians
    lat_rad = ee.Number(CENTER_LAT).multiply(math.pi / 180)
    ha_rad = ha.multiply(math.pi / 180)

    # Solar zenith angle (degrees)
    cos_zenith = lat_rad.sin().multiply(decl.sin()).add(
        lat_rad.cos().multiply(decl.cos()).multiply(ha_rad.cos())
    )
    
    zenith_rad = cos_zenith.acos()
    zenith_deg = zenith_rad.multiply(180 / math.pi)

    # Solar azimuth angle (degrees)
    cos_azimuth = (
        lat_rad.sin().multiply(cos_zenith).subtract(decl.sin())
    ).divide(
        lat_rad.cos().multiply(zenith_rad.sin())
    )
    
    # Todo: this might not be correct, maybe don't transform in degrees
    azimuth_deg = ee.Number(180).subtract(cos_azimuth.acos().multiply(180 / math.pi))

    # Hillshadow computation (terrain shadow mask)
    shadow = ee.Terrain.hillShadow(
        dsm_merc,
        azimuth_deg,
        zenith_deg
    ).rename('hillshadow')

    return image.addBands(shadow)

shadowed_collection = annotated.map(add_hillshadow_band) #TODO: maybe filter those images that don't have hillshadow after this

def log_missing_hillshadow(image):
    band_names = image.bandNames()
    return image.set({'has_hillshadow': band_names.contains('hillshadow')})

shadowed_collection = shadowed_collection.map(log_missing_hillshadow)
shadowed_collection = shadowed_collection.filter(ee.Filter.eq('has_hillshadow', True))

In [ ]:
problematic_id = '1_20200413T105621_20200413T105618_T32VKM_20200413T105621_20200413T110411_T32VKM_20200413T103948_20200415T062434'
# Test with one image 
test_img = shadowed_collection.filterDate('2020-04-14', '2020-05-01').first() #shadowed_collection.filter(ee.Filter.eq('system:index', problematic_id)).first()
test_img.bandNames().getInfo()
#TODO: fix error: , debug this script from top to bottom, try without landsat merge 
"""Error: reduce.mean: Error in 
map(ID=1_20200413T105621_20200413T105618_T32VKM_20200413T105621_20200413T110411_T32VKM_20200413T103948_20200415T062434): 
Image.select: Band pattern 'hillshadow' did not match any bands. Available bands: [cloud_mask] (Error code: 3)"""

['cloud_mask', 'hillshadow']

In [ ]:

# Visualize 
# Create interactive map centered on Bergen
Map = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=10, width=800, height=400)

# Visualize hillshadow band: 
hillshadow_vis = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'ffffff']  # white = sunlit, black = shadow
}

Map.addLayer(test_img.select('hillshadow'), hillshadow_vis, 'Hillshadow')
Map.addLayer(test_img.select('cloud_mask'), hillshadow_vis, 'Original Image')  # Optional: add base image

Map.addLayerControl()
Map

In [ ]:
# Parameter lists
def list_to_fc(val_list, prop_name='val'):
    return ee.FeatureCollection(
        val_list.map(lambda v: ee.Feature(None, {prop_name: v}))
    )

ALBEDO_VALUES_FC = list_to_fc(ee.List([0.081, 0.129, 0.174, 0.224, 0.354]))
COD_VALUES_FC = list_to_fc(ee.List([1.0, 3.41, 5.50, 7.68, 10.18, 13.67, 19.34, 27.79,
                                   42.03, 73.23, 125.42, 250.0]))
CBH_VALUES_FC = list_to_fc(ee.List([0.08, 0.167, 0.285, 0.571, 0.915, 1.286, 1.753,
                                   2.370, 3.171, 4.165, 5.451, 6.543, 8.498]))

# Define Fallback feature with default values to flag images where no LUT match was found
fallback_feature = ee.Feature(None, {
        'Diffuse_clear': -1,
        'Diffuse_cloudy': -1,
        'Direct_clear': -1,
        'Direct_cloudy': -1,
        'DOY': -999,
        'Hour': -1,
        'Albedo': -1,
        'Tau550': -1,
        'CloudBase_km': -1,
        'cloud_type_num': -1
    })
fallback_fc = ee.FeatureCollection([fallback_feature])

lut = ee.FeatureCollection("projects/sample-project-452812/assets/LUT")
# Filter only relevant hours for performance reasons
lut = lut.filter(ee.Filter.And(
    ee.Filter.gte('Hour', 9),
    ee.Filter.lte('Hour', 14)
))
# Add numeric cloud type (1 = ice, 0 = water) to LUT 
lut_ice = lut.filter(ee.Filter.eq('CloudType', 'ice')).map(lambda f : f.set('cloud_type_num', 1))
lut_water = lut.filter(ee.Filter.eq('CloudType', 'water')).map(lambda f : f.set('cloud_type_num', 0))
lut = lut_ice.merge(lut_water) # Test successful: number of values is still the same after filtering and merging
# Filter only relevant columns for performance reasons
lut = lut.select(['Hour', 'DOY', 'cloud_type_num', 'Albedo', 'CloudBase_km', 'Tau550',
                          'Direct_clear', 'Diffuse_clear', 'Direct_cloudy', 'Diffuse_cloudy'])


def find_closest(value, fc, prop_name='val'):
    # Add 'diff' property = abs(val - value)
    fc_with_diff = fc.map(lambda f: f.set('diff', ee.Number(f.get(prop_name)).subtract(value).abs()))
    # Sort by 'diff'
    sorted_fc = fc_with_diff.sort('diff')
    # Get closest feature
    closest = ee.Feature(sorted_fc.first())
    return ee.Number(closest.get(prop_name))

# Match each image to nearest LUT row
def match_to_lut(image):
    # Get raw input parameters
    doy = ee.Number(image.get('doy'))
    hour = ee.Number(image.get('hour')) 
    minute = ee.Number(image.get('minute'))
    # Round to closest full hour
    closest_hour = hour.add(minute.divide(60)).round()
    albedo = ee.Number(image.get('mean_albedo'))
    cod = ee.Number(image.get('mean_cod'))
    cbh = ee.Number(image.get('mean_cbh'))
    phase = ee.Number(image.get('cloud_type'))  # 1 ='ice' or 0='water'
    
    # Find closest values from LUT lists
    closest_albedo = find_closest(albedo, ALBEDO_VALUES_FC)
    closest_cod = find_closest(cod, COD_VALUES_FC)
    closest_cbh = find_closest(cbh, CBH_VALUES_FC)

    # Filter LUT
    filtered_lut = lut.filter(ee.Filter.And(
        ee.Filter.maxDifference(16, 'DOY', doy),
        ee.Filter.eq('Hour', closest_hour),
        ee.Filter.maxDifference(0.01, 'Albedo', closest_albedo), # use MaxDifference because there are numerical errors in LUT
        ee.Filter.maxDifference(0.01, 'Tau550', closest_cod),
        ee.Filter.maxDifference(0.01, 'CloudBase_km', closest_cbh),
        ee.Filter.eq('cloud_type_num', phase)
    ))
    
    # Append fallback feature to the filtered result, so .first() never returns null
    merged_lut = filtered_lut.merge(fallback_fc)

    # Get matching row (should be only one)
    matched = ee.Feature(merged_lut.first())

    # Copy values from LUT to image properties 
    return image.setMulti(matched.toDictionary([
        'Diffuse_clear', 'Diffuse_cloudy', 'Direct_clear', 'Direct_cloudy'
    ]))

with_lut = annotated.map(match_to_lut)

# Filter out masked images with Diffuse_clear = -1 --> no match found in LUT 
with_lut = with_lut.filter(ee.Filter.gte('Diffuse_clear', 0))

In [67]:
# Debug with problematic image
#problematic_id = '1_20250206T110301_20250206T110255_T32VKM_20250206T110301_20250206T110255_T32VKN_20250206T103238_20250208T021910'

#one_image = annotated.filter(ee.Filter.eq('system:index', problematic_id)).first()
one_image = annotated.filterDate('2020-04-15', '2020-05-01').first() 
debug_image = ee.Image(one_image)

# Extract image properties
# Get raw input parameters
doy = ee.Number(debug_image.get('doy'))
hour = ee.Number(debug_image.get('hour')) 
minute = ee.Number(debug_image.get('minute'))
# Round to closest full hour
closest_hour = hour.add(minute.divide(60)).round()
albedo = ee.Number(debug_image.get('mean_albedo'))
cod = ee.Number(debug_image.get('mean_cod'))
cbh = ee.Number(debug_image.get('mean_cbh'))
phase = ee.Number(debug_image.get('cloud_type'))

print("DOY: ", doy.getInfo())
print("Closest_hour: ", closest_hour.getInfo())
print("Mean albedo: ", albedo.getInfo())
print("Mean cod: ", cod.getInfo())
print("Mean cbh: ", cbh.getInfo())
print("cloud phase: ", phase.getInfo())

# Snap values
# Find closest values from LUT lists
closest_albedo = find_closest(albedo, ALBEDO_VALUES_FC)
closest_cod = find_closest(cod, COD_VALUES_FC)
closest_cbh = find_closest(cbh, CBH_VALUES_FC)

#print("closest albedo: ", closest_albedo.getInfo())
#print("closest cod: ", closest_cod.getInfo())
#print("closest cbh: ", closest_cbh.getInfo())

# Filter LUT and inspect how many matches
filtered_lut = lut.filter(ee.Filter.And(
        ee.Filter.maxDifference(16, 'DOY', doy),
        ee.Filter.eq('Hour', closest_hour),
        ee.Filter.maxDifference(0.01, 'Albedo', closest_albedo), # use MaxDifference because there are numerical errors in LUT
        ee.Filter.maxDifference(0.01, 'Tau550', closest_cod),
        ee.Filter.maxDifference(0.01, 'CloudBase_km', closest_cbh),
        ee.Filter.eq('cloud_type_num', phase)
))
    
# Append fallback feature to the filtered result, so .first() never returns null
merged_lut = filtered_lut.merge(fallback_fc)

print("LUT match count:", merged_lut.size().getInfo())

# View feature if one exists
matched = ee.Feature(merged_lut.first())
print("Matched LUT feature:", matched.toDictionary().getInfo())

DOY:  109
Closest_hour:  11
Mean albedo:  0.2561546607878025
Mean cod:  7.805837991496782
Mean cbh:  0.5555487452223897
cloud phase:  0
LUT match count: 2
Matched LUT feature: {'Albedo': 0.2240000069141388, 'CloudBase_km': 0.5709999799728394, 'DOY': 105, 'Diffuse_clear': 99.93006896972656, 'Diffuse_cloudy': 357.7716064453125, 'Direct_clear': 523.21728515625, 'Direct_cloudy': 0.0021170200780034065, 'Hour': 11, 'Tau550': 7.679999828338623, 'cloud_type_num': 0}


In [89]:
# Compute GHI from cloud and hillshadow logic
def compute_ghi(image):
    # Constants from LUT
    dir_clear = ee.Number(image.get('Direct_clear'))
    dif_clear = ee.Number(image.get('Diffuse_clear'))
    dir_cloudy = ee.Number(image.get('Direct_cloudy'))
    dif_cloudy = ee.Number(image.get('Diffuse_cloudy'))

    cloud_mask = image.select('cloud_mask')  # 1 = cloudy
    hillshadow = image.select('hillshadow')  # 1 = in sunlight

    ghi_direct = cloud_mask.eq(1).multiply(dir_cloudy).add(
                   cloud_mask.eq(0).multiply(dir_clear))

    ghi_diffuse = cloud_mask.eq(1).multiply(dif_cloudy).add(
                    cloud_mask.eq(0).multiply(dif_clear))

    ghi_total = hillshadow.eq(0).multiply(ghi_diffuse).add(
                  hillshadow.eq(1).multiply(ghi_direct.add(ghi_diffuse))) # 1=sunlight

    # Ensure consistent float type and band names
    #ghi_direct = ghi_direct.toFloat().rename('GHI_direct')
    #ghi_diffuse = ghi_diffuse.toFloat().rename('GHI_diffuse')
    ghi_total = ghi_total.toFloat().rename('GHI_total')

    # For memory optimization, only add ghi_total and remove all other bands 
    return image.addBands([ghi_total], overwrite=True).select('GHI_total')
    

with_ghi = with_lut.map(compute_ghi)

In [ ]:
test_img = shadowed_collection.first() 
test_img_with_lut = match_to_lut(ee.Image(test_img))
test_img_with_ghi = compute_ghi(ee.Image(test_img_with_lut))

# Visualize 
# Create interactive map centered on Bergen
Map = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=10,height=400, width=800)

# Visualize ghi band: blue = no sunlight, yellow = a lot of sunlight
ghi_vis = {
    'min': 0,
    'max': 1000,
    'palette': ['blue', 'green', 'yellow']  # white = sunlit, black = shadow
}


Map.addLayer(test_img.select('hillshadow'), hillshadow_vis, 'Hillshadow')
Map.addLayer(test_img_with_ghi.select('GHI_total'), ghi_vis, 'GHI Total')  
Map.addLayer(test_img.select('cloud_mask'), hillshadow_vis, 'Original Image') 

Map.addLayerControl()
Map

Map(center=[60.39, 5.33], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

Aggregate and export months, seasons and alltime aggregate
Image count per month: (merged_all)
Month 1: 324 images
Month 2: 360 images
Month 3: 438 images
Month 4: 542 images
Month 5: 507 images
Month 6: 509 images
Month 7: 448 images
Month 8: 480 images
Month 9: 383 images
Month 10: 430 images
Month 11: 387 images
Month 12: 124 images

In [ ]:
scale = 30

def export_img(image, filename_prefix):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename_prefix,
        folder='GEE_exports',
        fileNamePrefix=filename_prefix,
        region=bergen_roi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print(f"Export task for image '{filename_prefix}' has been scheduled.")


# Seasonal aggregates 
SEASONS = {
    'Winter': [12, 1, 2],
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Autumn': [9, 10, 11]
}

# Export monthly mean images
for m in range(1, 13):
    filtered_month = with_ghi.filter(ee.Filter.eq('month', m))
    mean_img = filtered_month.select('GHI_total').mean()
    month_name = calendar.month_name[m]  # e.g., January, February, ...
    filename = f"surface_ghi_mean_{month_name}_scale_{scale}"
    #export_img(mean_img, filename)
    
# Export seasonal mean images
for season in SEASONS:
    months = ee.List(SEASONS[season])
    filtered = with_ghi.filter(ee.Filter.inList('month', months))
    mean_img = filtered.select('GHI_total').mean()
    filename = f"surface_ghi_mean_{season}_scale_{scale}"
    #export_img(mean_img, filename)
    

# Export alltime 
ghi_alltime = with_ghi.select('GHI_total').mean()
#export_img(ghi_alltime, f"surface_ghi_mean_April2020_scale_{scale}") 

Export task for image 'surface_ghi_mean_April2020_scale_30' has been scheduled.


In [91]:
ghi_vis = {
    'min': 0,
    'max': 1000,
    'palette': ['blue', 'green', 'yellow']  # white = sunlit, black = shadow
}

Map = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=10,height=400, width=800)
Map.addLayer(ghi_alltime, ghi_vis, 'GHI') #6s runtime with 24 images, 1-2min with 122 images (1 month)
Map

KeyboardInterrupt: 

In [ ]:
# Reduce the image to a histogram dictionary over the region
hist_dict = june_img.reduceRegion(
    reducer=ee.Reducer.histogram(maxBuckets=255),
    geometry=bergen_roi,
    scale=30,
    bestEffort=True,
    maxPixels=1e8
).get('GHI_total').getInfo()

# Extract histogram data
hist_values = hist_dict['histogram']
bucket_means = hist_dict['bucketMeans']

# Plot histogram
plt.figure(figsize=(10, 5))
plt.bar(bucket_means, hist_values, width=hist_dict['bucketWidth'], align='center', color='skyblue')
plt.title('Histogram of GHI_alltime Pixel Values')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

EEException: User memory limit exceeded.